# RTEM Hackathon notebook

## Import standard libraries

In [13]:
import pandas as pd
import json
import numpy as np

## Import RTEM libraries

In [5]:
# Install onboard client to python, uncomment and run once
'''
!pip install onboard.client
'''

'\n!pip install onboard.client\n'

In [18]:
from onboard.client import OnboardClient
from onboard.client.dataframes import points_df_from_streaming_timeseries
from onboard.client.models import PointSelector, TimeseriesQuery, PointData
from datetime import datetime, timezone, timedelta
from typing import List
import pytz

## Load secret

In [19]:
# Read secret from /secrets/ folder
with open('secrets/secrets.txt') as f:
    secret = f.readlines()[0]

# Setup client
client = OnboardClient(api_key=secret)

In [20]:
print(list(pd.DataFrame(client.get_all_buildings())['name'])) # returns list of buildings that you have access to (you may not have 'Laboratory' in your set)

['88737', '83483', '94414', '89496', '116742', '90366', '129666', '80324', '94400', '94412', '94411', '128286', '116399', '97789', '164423', '76513', '124540', '87722', '114460', '139252', '164994', '79789', '84869', '145732', '152000', '78797', '153255', '130262', '150388', '73107', '110897', '159950', '149911', '159109', '116350', '160783', '85481', '133932', '131034', '164864', '111479', '89089', '144601', '116723', '128615', '137940', '163896', '113413', '68855', '163716', '164033', '137257', '112620', '128313', '104698', '87261', '99315', '158818', '96938', '90367', '124556', '130079', '75216', '108774', '122677', '144540', '82119', '114249', '145813', '92344', '136558', '86328', '108998', '155776', '75620', '124529', '75049', '119313', '122607', '89522', '88776', '137244', '123183', '92894', '93400', '141633', '116888', '117460', '73582', '111473', '68975', '77439', '75506', '79790', '113572', '89088', '94395', '125921', '103083', '89097', '130853', '128312', '86373', '88777', '1

In [22]:
query = PointSelector()
query.point_types     = ['Zone Temperature'] # can list multiple point
query.equipment_types = ['fcu']              # types, equipment types,
query.buildings       = ['Laboratory']       # buildings, etc.
selection = client.select_points(query)

start = datetime.now(pytz.utc) - timedelta(days=7)
end   = datetime.now(pytz.utc)

timeseries_query = TimeseriesQuery(point_ids = selection['points'], start = start, end = end)
sensor_data = points_df_from_streaming_timeseries(client.stream_point_timeseries(timeseries_query))

ValidationError: 1 validation error for TimeseriesQuery
point_ids
  Exactly one of 'point_ids' or 'selector' is required (type=value_error)